In [31]:
# https://www.kaggle.com/jsphyg/weather-dataset-rattle-package 
import numpy as np
import pandas as pd

# Wczytujemy plik "weatherAUS.csv" przy użyciu bilioteki pandas
df = pd.read_csv('weatherAUS.csv')

# Wypisujemy wymiary wczytanego pliku
print('Wymiary wczytanego pliku:',df.shape)

# Wypisujemy pierwsze 5 wierszy DataFrame.
df[0:5]

('Wymiary wczytanego pliku:', (142193, 24))


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [32]:
# Sprawdźmy które kolumny mają dużo "missing values".
# Żeby dobrze zrozumieć co się dzieje w tej linijce warto wypisać sobie poszczególne
# wartości oddzielnie, czyli:
# df.isnull()
# df.isnull().sum()
df.isnull().sum().sort_values()

Date                 0
RISK_MM              0
RainTomorrow         0
Location             0
MaxTemp            322
MinTemp            637
Temp9am            904
WindSpeed9am      1348
Rainfall          1406
RainToday         1406
Humidity9am       1774
WindSpeed3pm      2630
Temp3pm           2726
Humidity3pm       3610
WindDir3pm        3778
WindGustSpeed     9270
WindGustDir       9330
WindDir9am       10013
Pressure3pm      13981
Pressure9am      14014
Cloud9am         53657
Cloud3pm         57094
Evaporation      60843
Sunshine         67816
dtype: int64

In [33]:
# Usuwamy 4 kolumny które mają mniej niż 60 % danych
# Usuwamy też, "location", bo chcemy aby nasz model przewidywał 
# dane nizależnie dla całej Australii
# Usuwamy RISK_MM poniważ jest to ściśle powiązane z 'RainTomorrow'
# Na stronie https://www.kaggle.com/jsphyg/weather-dataset-rattle-package możemy przeczytać definicję RISK_MM:
# RISK_MM - The amount of rain. A kind of measure of the "risk".
df = df.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','Location','RISK_MM','Date'],axis=1)

# Wypisujemy wymiary "df" po usunięciu 7 kolumn, z 24 zostało ich 17
df.shape

(142193, 17)

In [34]:
# Usuwamy wszystkie NaN (not a number)
df = df.dropna(how='any')
# Wypisujemy wymiary "df" po usunięciu NaN z 142193 wierszy zostało 112925
df.shape

(112925, 17)

In [35]:
# Zamieniamy "Yes" i "no" na wartości liczbowe
df['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
df['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)

# Zamiana kategorycznych wartości przy użyciu pd.getDummies()
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
categorical_columns = ['WindGustDir', 'WindDir3pm', 'WindDir9am']
for col in categorical_columns:
    print(np.unique(df[col]))
df = pd.get_dummies(df, columns=categorical_columns)

# Wypisujemy pierwsze 5 wierszy DataFrame. Zwróćmy uwagę na kolumny:
# "WindDir9am_NNW", "WindDir9am_NW", "WindDir9am_S" etc...
# jest to efekt działania funkcji get_dummies(), zamienia ona "stringi" czyli napisy na
# wartości liczbowe. Więcej na:
# https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f
df.iloc[0:5]

['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']
['E' 'ENE' 'ESE' 'N' 'NE' 'NNE' 'NNW' 'NW' 'S' 'SE' 'SSE' 'SSW' 'SW' 'W'
 'WNW' 'WSW']


,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,0,0,0,0,0,0,0,1,0,0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,1,0,0,0,0,0,0,0,0,0
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,0,0,0,0,0,0,0,1,0,0
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,0,0,0,1,0,0,0,0,0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Normalizujemy dane przy użyciu MinMaxScaler()

from sklearn import preprocessing

# W tej linijce tylko tworzymy MinMaxScaler()
scaler = preprocessing.MinMaxScaler()

# Tutaj przekazujemy do Scalera nasze dane i scaler analizuje wszystkie wartości 
scaler.fit(df)

# W tej lini następuje faktyczne przekształcenie danych.
df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)
# Wypisujemy pierwsze 5 wierszy DataFrame. Zwróćmy uwagę na to jak zmieniły się wartości, na przykład "MinTemp".
df.iloc[0:5]

In [37]:
# Tworzymy nasz "input" i "target"
X = df.loc[:,df.columns!='RainTomorrow']
y = df[['RainTomorrow']]

**Finding the best model**

In [38]:
from sklearn.model_selection import train_test_split
# Dzielimy dane na train i test set (75% to train set, zaś 25% to test set).
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
print(X_train.shape)
print(X_test.shape)

(84693, 61)
(28232, 61)


In [39]:
#Logistic Regression 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

# Rozpoczynamy pomiar czasu
t0=time.time()

# Tworzymy model LogisticRegression
clf_logreg = LogisticRegression(random_state=0)

# Trenujemy model na danych treningowych
clf_logreg.fit(X_train,y_train)

# Dokonujemy predykcji dla danych testowych.
y_pred = clf_logreg.predict(X_test)

# Obliczamy dokładność dla danych testowych
score = accuracy_score(y_test,y_pred)
print('Accuracy :',score)

# Wypisujemy czas który minął
print('Time taken :' , time.time()-t0)


('Accuracy :', 0.8484344006800794)
('Time taken :', 0.8686830997467041)


In [40]:
#Random Forest Classifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Rozpoczynamy pomiar czasu
t0=time.time()

# Tworzymy model RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=0, min_samples_leaf=2, criterion="gini")

# Trenujemy model na danych treningowych
clf_rf.fit(X_train,y_train)

# Dokonujemy predykcji dla danych testowych.
y_pred = clf_rf.predict(X_test)

# Obliczamy dokładność dla danych testowych
score = accuracy_score(y_test,y_pred)
print('Accuracy :',score)

# Wypisujemy czas który minął
print('Time taken :' , time.time()-t0)

/home/kuba/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


('Accuracy :', 0.8437588551997733)
('Time taken :', 0.5768001079559326)


In [41]:
#Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Rozpoczynamy pomiar czasu
t0=time.time()

# Tworzymy model DecisionTreeClassifier
clf_dt = DecisionTreeClassifier(random_state=0)

# Trenujemy model na danych treningowych
clf_dt.fit(X_train,y_train)

# Dokonujemy predykcji dla danych testowych.
y_pred = clf_dt.predict(X_test)

# Obliczamy dokładność dla danych testowych
score = accuracy_score(y_test,y_pred)
print('Accuracy :',score)

# Wypisujemy czas który minął
print('Time taken :' , time.time()-t0)

('Accuracy :', 0.7868022102578635)
('Time taken :', 1.113832950592041)
